In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
!git clone https://www.github.com/emanhamed/Houses-dataset

In [ ]:
import keras
import cv2
from sklearn.preprocessing import LabelBinarizer,StandardScaler,MinMaxScaler
from keras.layers import Dense,Activation,Conv2D,BatchNormalization,Dropout,MaxPool2D,Flatten
from sklearn.model_selection import train_test_split

In [ ]:
cd Houses-dataset/'Houses Dataset'

In [ ]:
cols = ['Bedroom','Bathroom','Area','Zip','Price']

In [ ]:
df = pd.read_csv('HousesInfo.txt',header = None,sep = ' ',names = cols)
df.head()

In [ ]:
cat = ['Bathroom','Bedroom','Area']

In [ ]:
mm = MinMaxScaler()
lb = LabelBinarizer()

In [ ]:
zips = df['Zip'].value_counts().keys().tolist()

In [ ]:
occ = df['Zip'].value_counts().tolist()

In [ ]:
zip_data = pd.DataFrame()
zip_data['Zip'] = zips
zip_data['occ'] = occ

In [ ]:
keep_ind = []
del_ind = []

In [ ]:
for i in range(len(df)):
    zc = df['Zip'][i]
    id = zip_data[zip_data['Zip']==zc].index.tolist()
    if zip_data['occ'][id[0]]>20:
        keep_ind.append(i)
    else:
        del_ind.append(i)

In [ ]:
df.drop(del_ind,inplace = True)
df.head(10)

In [ ]:
df[cat] = mm.fit_transform(df[cat])

In [ ]:
X  = df.drop(columns = ['Price'])
Y = df['Price']

In [ ]:
train_x,test_x,train_y,test_y = train_test_split(X,Y,test_size = 0.2,random_state = 10)

In [ ]:
train_zip = lb.fit_transform(train_x['Zip'])
test_zip = lb.fit_transform(test_x['Zip'])

In [ ]:
train_x.drop(columns = ['Zip'],inplace = True)

In [ ]:
test_x.drop(columns = ['Zip'],inplace = True)

In [ ]:
train_x_new = np.hstack([train_x,train_zip])
test_x_new = np.hstack([test_x,test_zip])

In [ ]:
train_y = np.array(train_y)
test_y = np.array(test_y)

In [ ]:
train_y = train_y.reshape((-1,1))
test_y = test_y.reshape((-1,1))

In [ ]:
def nn(op):
    model = keras.models.Sequential()
    model.add(Dense(16,input_dim = 11,activation = 'relu'))
    model.add(Dense(8,activation = 'relu'))
    model.add(Dense(4,activation = 'relu'))
    if op:
        model.add(Dense(1,activation = 'linear'))
    return model

In [ ]:
def cnn(op):
    model = keras.models.Sequential()
    
    ip = keras.models.Input((256,256,3))
    
    x = Conv2D(16,kernel_size=(3,3),strides=(1,1),padding = 'same')(ip)
    x = Activation('relu')(x)
    x = BatchNormalization(axis = -1)(x)
    x = MaxPool2D(pool_size = (2,2))(x)
    
    x = Conv2D(32,kernel_size=(3,3),strides=(1,1),padding = 'same')(x)
    x = Activation('relu')(x)
    x = BatchNormalization(axis = -1)(x)
    x = MaxPool2D(pool_size = (2,2))(x)
    
    x = Conv2D(64,kernel_size=(3,3),strides=(1,1),padding = 'same')(x)
    x = Activation('relu')(x)
    x = BatchNormalization(axis = -1)(x)
    x = MaxPool2D(pool_size = (2,2))(x)
    
    x = Conv2D(128,kernel_size=(3,3),strides=(1,1),padding = 'same')(x)
    x = Activation('relu')(x)
    x = BatchNormalization(axis = -1)(x)
    x = MaxPool2D(pool_size = (2,2))(x)
    
    x = Flatten()(x)
    
    x = Dense(32,activation='relu')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Dropout(0.35)(x)
    
    x = Dense(16,activation='relu')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Dropout(0.35)(x)
    
    x = Dense(8,activation='relu')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Dropout(0.4)(x)
    
    x = Dense(4,activation='relu')(x)
    
    if op:
        x = Dense(1,activation='linear')(x)
        
    model = keras.models.Model(inputs = ip,outputs = x)
    
    return model

In [ ]:
images = []

In [ ]:
for i in range(len(df)):
  id = df.index[i]
  
  path1 = str(id+1)+'_frontal.jpg'
  i1 = cv2.imread(path1)
  i1 = cv2.resize(i1,(128,128))
  i1 = cv2.cvtColor(i1,cv2.COLOR_BGR2RGB)
  
  path2 = str(id+1)+'_bathroom.jpg'
  i2 = cv2.imread(path2)
  i2 = cv2.resize(i2,(128,128))
  i2 = cv2.cvtColor(i2,cv2.COLOR_BGR2RGB)
  
  path3 = str(id+1)+'_bedroom.jpg'
  i3 = cv2.imread(path3)
  i3 = cv2.resize(i3,(128,128))
  i3 = cv2.cvtColor(i3,cv2.COLOR_BGR2RGB)
  
  path4 = str(id+1)+'_kitchen.jpg'
  i4 = cv2.imread(path4)
  i4 = cv2.resize(i4,(128,128))
  i4 = cv2.cvtColor(i4,cv2.COLOR_BGR2RGB)
  
  i = np.zeros((256,256,3),dtype = 'uint8')
  i[0:128,0:128] = i1
  i[128:256,128:256] = i2
  i[0:128,128:256] = i3
  i[128:256,0:128] = i4
  
  images.append(i)

In [ ]:
images = np.array(images)
images.shape

In [ ]:
train_df,test_df,train_images,test_images = train_test_split(df,images,test_size = 0.2,random_state = 10)

In [ ]:
train_zip2 = lb.fit_transform(train_df['Zip'])
test_zip2 = lb.fit_transform(test_df['Zip'])

In [ ]:
train_y2 = train_df['Price']
test_y2 = test_df['Price']
mxm = train_y2.max()
train_y2 = train_y2/mxm
test_y2 = test_y2/mxm

In [ ]:
train_df.drop(columns = ['Zip','Price'],inplace = True)
test_df.drop(columns = ['Zip','Price'],inplace = True)

In [ ]:
train_x2 = np.hstack([train_df,train_zip2])
test_x2 = np.hstack([test_df,test_zip2])

In [ ]:
nn_model = nn(op = False)
cnn_model = cnn(op = False)

In [ ]:
opt = keras.optimizers.adam(lr = 0.001,decay = 0.00000555555)

In [ ]:
ip_cat = m=keras.layers.concatenate([nn_model.output,cnn_model.output])

In [ ]:
X = Dense(4,activation = 'relu')(ip_cat)
X = Dense(1,activation='linear')(X)

In [ ]:
final_model = keras.models.Model(inputs = [nn_model.input,cnn_model.input],outputs = X)

In [ ]:
final_model.compile(optimizer = opt,loss = 'mean_absolute_percentage_error')

In [ ]:
train_y2 = np.array(train_y2)

In [ ]:
train_y2 = train_y2.reshape((-1,1))

In [ ]:
test_y2 = np.array(test_y2)
test_y2 = test_y2.reshape((-1,1))

In [ ]:
final_model.fit([train_x2,train_images],train_y2,validation_data=([test_x2,test_images],test_y2),epochs = 150,batch_size = 8)

In [ ]:
final_pred = final_model.predict([test_x2,test_images])

In [ ]:
np.mean(np.abs(((test_y2-final_pred)/test_y2)*100))

In [ ]:
cd ..

In [ ]:
cd ..

In [ ]:
rm -rf Houses-dataset/

In [ ]:
final_model.save_weights('final.h5')